In [2]:
import torch
import numpy as np

## 3.3 Constructing Tensor

In [44]:
x = torch.rand(4, 3, dtype=torch.float32)
x

tensor([[0.0205, 0.5319, 0.9332],
        [0.8305, 0.0015, 0.8126],
        [0.7429, 0.9841, 0.5914],
        [0.6943, 0.3579, 0.8555]])

In [45]:
x[1]

tensor([0.8305, 0.0015, 0.8126])

In [49]:
# 2nd and 3rd row
x[1:3, :]

tensor([[0.8305, 0.0015, 0.8126],
        [0.7429, 0.9841, 0.5914]])

In [47]:
# 2nd column
x[:, 1]

tensor([0.5319, 0.0015, 0.9841, 0.3579])

In [51]:
# 2nd row and 2 column
x[1, 1]

tensor(0.0015)

## 3.4 Named Tensors

### Create named vector

In [18]:
x = torch.rand(32, 2, 28, 28)
x = x.refine_names(..., 'colors', 'width', 'height')
x.shape, x.names

(torch.Size([32, 2, 28, 28]), (None, 'colors', 'width', 'height'))

In [23]:
x = torch.rand(32, 3, 28, 28, names=['batch_size', 'colors', 'width', 'height'])
x.shape, x.names

(torch.Size([32, 3, 28, 28]), ('batch_size', 'colors', 'width', 'height'))

### run operations

In [28]:
weights = torch.rand(32, 1, names=['batch_size', 'colors'])
weights.shape, weights.names

(torch.Size([32, 1]), ('batch_size', 'colors'))

In [30]:
weights_aligned = weights.align_as(x)
weights_aligned.shape, weights_aligned.names

(torch.Size([32, 1, 1, 1]), ('batch_size', 'colors', 'width', 'height'))

In [33]:
mul = (x * weights_aligned).sum('colors')
mul.shape, mul.names

(torch.Size([32, 28, 28]), ('batch_size', 'width', 'height'))

In [35]:
mul = mul.rename(None)
mul.shape, mul.names

(torch.Size([32, 28, 28]), (None, None, None))

## 3.5 Tensor element types

### 3.5.2 indexing using tensor

In [58]:
ix = torch.Tensor([2, 2])
ix = ix.type(torch.int64) # this dtype is mandatory
x[ix]

tensor([[0.7429, 0.9841, 0.5914],
        [0.7429, 0.9841, 0.5914]])

### 3.5.3 ways of setting dtype

In [60]:
x = torch.ones(2, 2, dtype=torch.float)

In [61]:
x = torch.ones(2, 2).float()

In [63]:
x = torch.ones(2, 2).to(dtype=torch.float)

In [64]:
# dtypes are automatically cast to bigger during operations
x_32 = torch.rand(5, dtype=torch.float)
x_64 = torch.rand(5, dtype=torch.double)
(x_32 * x_64).dtype

torch.float64

## 3.6 the Tensor API

Operations on the tensor can be both coded as function of tensor module or method of a tensor

In [10]:
a = torch.ones(1, 6)

b = torch.transpose(a, 0, 1)
a.shape, b.shape

(torch.Size([1, 6]), torch.Size([6, 1]))

In [13]:
b = a.transpose(0, 1)
a.shape, b.shape

(torch.Size([1, 6]), torch.Size([6, 1]))

## 3.8 Tensor metadata: Size, offset, and stride

### 3.8.1 Views of another tensor’s storage

In [18]:
points = torch.rand(5, 2)
points

tensor([[0.9064, 0.7384],
        [0.4312, 0.6706],
        [0.6488, 0.2602],
        [0.9775, 0.6646],
        [0.9423, 0.7460]])

In [19]:
# Changes value of subtensor will change value in original tensor
second_point = points[1]
second_point[0] = 1
points

tensor([[0.9064, 0.7384],
        [1.0000, 0.6706],
        [0.6488, 0.2602],
        [0.9775, 0.6646],
        [0.9423, 0.7460]])

In [20]:
# if you create a copy it will allocate new storage for this suntensor
third_point = points[2].clone()
third_point[0] = 1
third_point, points

(tensor([1.0000, 0.2602]),
 tensor([[0.9064, 0.7384],
         [1.0000, 0.6706],
         [0.6488, 0.2602],
         [0.9775, 0.6646],
         [0.9423, 0.7460]]))

## 3.10 NumPy itneroperability

In [24]:
arr = np.random.rand(3, 3)
id(arr)

5376411760

In [25]:
arr_tensor = torch.from_numpy(arr)
id(arr_tensor)

5379030000

## 3.12 Serializing Tensor

In [26]:
torch.save(arr_tensor, 'data/arr-tensor.t')

In [27]:
arr_tensor_loaded = torch.load('data/arr-tensor.t')

### 3.12.1 Serializing to HDF5 with h5py

In [29]:
import h5py

In [35]:
points_tensor = torch.rand(100, 100)
points_numpy = points_tensor.numpy()

In [38]:
# save matrix as file

In [34]:
points_file = h5py.File(name='data/random_point_coord.h5', mode='w') 

In [36]:
points_file.create_dataset(name='coords', data=points_numpy)

<HDF5 dataset "coords": shape (100, 100), type "<f4">

In [37]:
points_file.close()

In [39]:
# read elements of matrix without loading whole matri

In [40]:
points_file = h5py.File(name='data/random_point_coord.h5', mode='r')
points = points_file['coords']

In [42]:
# Only ow we read part of matrix into memory
points[69, 69]

0.017864108

In [43]:
points_file.close()

## 3.14 Exercise

In [44]:
a = torch.from_numpy(np.array(list(range(9))))

In [54]:
a

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [60]:
a.storage()

/var/folders/pc/0ygcdhb96px_y8nf8nzzgr8m0000gn/T/ipykernel_60517/214256462.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  a.storage()


 0
 1
 2
 3
 4
 5
 6
 7
 8
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 9]

In [47]:
a.size(), a.stride(), a.storage_offset()

(torch.Size([9]), (1,), 0)

In [61]:
b = a.view(3, 3)
b.size(), b.stride(), b.storage_offset()

(torch.Size([3, 3]), (3, 1), 0)

In [55]:
b

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

In [50]:
id(a),id(b)

(5488802480, 5381111952)

In [62]:
c = b[1:, 1:]
id(c), c.size(), c.stride(), c.storage_offset()

(5376058736, torch.Size([2, 2]), (3, 1), 4)

In [63]:
torch.sqrt(a)

tensor([0.0000, 1.0000, 1.4142, 1.7321, 2.0000, 2.2361, 2.4495, 2.6458, 2.8284])

In [64]:
a.sqrt()

tensor([0.0000, 1.0000, 1.4142, 1.7321, 2.0000, 2.2361, 2.4495, 2.6458, 2.8284])

In [65]:
a.sqrt_()

RuntimeError: result type Float can't be cast to the desired output type Long